# Exploring the UTx000 Extension Beacon Data
(Known as BPEACE2 in the [GH repo](https://github.com/intelligent-environments-lab/utx000))

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Evening IEQ Measurements
Now that we have parsed the beacon data into time periods corresponding to when participants were asleep, it would be interesting to analyze what concentrations are being experienced during the evening.

## Package Import

In [2]:
import os
import sys
sys.path.append('../')

from src.features import build_features
from src.visualization import visualize
from src.reports import make_report

import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# Data Import
We are only concerned with the beacon data when participants are asleep since that is the only time we can be sure participants are home in their bedrooms.

In [ ]:
build_features.get_restricted_beacon_datasets(restrict_by_ema=False)

In [ ]:
beacon = pd.read_csv('../data/processed/bpeace2-beacon-fb_and_gps_restricted.csv',
                    index_col=0,parse_dates=[0,-1,-2],infer_datetime_format=True)
beacon.head()

# Finding Nights with Constant CO2 Concentration
We need to find nights when the CO2 concentration is constant for at least an hour.

In [18]:
good_nights = pd.DataFrame()
for pt in beacon['Beiwe'].unique():
    beacon_pt = beacon[beacon['Beiwe'] == pt]
    for start, end in zip(beacon_pt['start_time'],beacon_pt['end_time']):
        if (end - start).total_seconds()/3600 > 5: # sleeping longer than 5 hours
            beacon_pt_night = beacon_pt[start:end] # iaq data during sleep
            std = np.std(beacon_pt_night)
            diffs = [0]
            for i in range(len(beacon_pt_night)-1):
                diff = abs(beacon_pt_night['CO2'][i+1] - beacon_pt_night['CO2'][i])
                diffs.append(diff)
            beacon_pt_night['differences'] = diffs
            no_change = beacon_pt_night[beacon_pt_night['differences'] < std]
            num_mins = []
            for i in range(len(no_change)):
                dts = []
                

ValueError: Can only compare identically-labeled Series objects

In [17]:
beacon_pt_night

NameError: name 'beacon_pt_night' is not defined